In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import shfl
import torch
import copy
import cv2
import numpy as np

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv, get_data_csv
from ClassifierModel import ClassifierModel

In [44]:
def cit_builder():    
    return CITModel(['N', 'P'], classifier_name = "resnet18", folds=1, lambda_values = [0.05], batch_size=1, epochs=1, device='cpu')


In [45]:
cit1 = cit_builder()
cit2 = cit_builder()
cit3 = cit_builder()

cit2._G_dict['P'].to('cpu')

#for p in cit2._G_dict['P'].parameters():
#    print(p)

state_dict = cit2._G_dict['P'].state_dict()
new_state_dict = copy.deepcopy(state_dict)

for k in state_dict:
    print(state_dict[k])


tensor([[[[ 0.0265, -0.0154, -0.0171,  ...,  0.0523, -0.0348,  0.0611],
          [ 0.0490,  0.0238, -0.0189,  ...,  0.0213, -0.0585,  0.0423],
          [ 0.0045,  0.0639,  0.0339,  ..., -0.0182,  0.0080,  0.0012],
          ...,
          [ 0.0201, -0.0094,  0.0003,  ..., -0.0332, -0.0601, -0.0578],
          [ 0.0274, -0.0510, -0.0419,  ...,  0.0271,  0.0259, -0.0244],
          [-0.0372, -0.0502, -0.0361,  ..., -0.0267,  0.0274,  0.0160]],

         [[-0.0515, -0.0228, -0.0158,  ..., -0.0384,  0.0179,  0.0441],
          [ 0.0354, -0.0315,  0.0388,  ..., -0.0042, -0.0014, -0.0111],
          [ 0.0521,  0.0344, -0.0585,  ...,  0.0208, -0.0530,  0.0407],
          ...,
          [-0.0436, -0.0412, -0.0596,  ..., -0.0602,  0.0153, -0.0038],
          [ 0.0503, -0.0562,  0.0610,  ...,  0.0420,  0.0356,  0.0599],
          [-0.0101,  0.0200, -0.0295,  ..., -0.0333,  0.0198,  0.0014]],

         [[-0.0248, -0.0285,  0.0422,  ..., -0.0254,  0.0190,  0.0597],
          [ 0.0265, -0.0350,  

In [46]:
with torch.no_grad():
    #for param1, param2 in zip(cit1._G_dict['P'].parameters(), cit2._G_dict['P'].parameters()):
    #    param2.copy_(param1.data - param2.data)
    for k in state_dict:
        #print(k)
        new_state_dict[k] = 2*state_dict[k]
        #print(new_state_dict[k])

cit2._G_dict['P'].load_state_dict(new_state_dict)
    
    

In [47]:
state_dict = cit2._G_dict['P'].state_dict()
for k in state_dict:
    print(state_dict[k])

tensor([[[[ 0.0530, -0.0308, -0.0341,  ...,  0.1046, -0.0695,  0.1222],
          [ 0.0980,  0.0477, -0.0377,  ...,  0.0427, -0.1171,  0.0847],
          [ 0.0090,  0.1277,  0.0677,  ..., -0.0365,  0.0159,  0.0025],
          ...,
          [ 0.0402, -0.0188,  0.0006,  ..., -0.0664, -0.1201, -0.1156],
          [ 0.0548, -0.1020, -0.0837,  ...,  0.0543,  0.0519, -0.0488],
          [-0.0744, -0.1004, -0.0722,  ..., -0.0535,  0.0549,  0.0320]],

         [[-0.1031, -0.0456, -0.0316,  ..., -0.0769,  0.0358,  0.0881],
          [ 0.0708, -0.0630,  0.0777,  ..., -0.0085, -0.0027, -0.0221],
          [ 0.1042,  0.0689, -0.1170,  ...,  0.0416, -0.1061,  0.0814],
          ...,
          [-0.0872, -0.0824, -0.1192,  ..., -0.1204,  0.0307, -0.0077],
          [ 0.1007, -0.1125,  0.1220,  ...,  0.0840,  0.0712,  0.1198],
          [-0.0201,  0.0399, -0.0589,  ..., -0.0667,  0.0396,  0.0028]],

         [[-0.0495, -0.0569,  0.0843,  ..., -0.0508,  0.0379,  0.1194],
          [ 0.0530, -0.0701,  